<left>FINM 33150 - Quantitative Trading Strategies</left>
<left>Winter 2023</left>
<br>
<h1><center> Homework 3: Simple Spread Trading </center></h1>
<center>Due - 23:00 [CST] January 26th, 2023</center>
<br>
<h3>Ki Hyun</h3>
<h3>Student ID: 12125881</h3>

<h5> Imports </h5>

In [1]:
%matplotlib inline

In [2]:
import os
from datetime import datetime, timedelta
import quandl
import functools

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns

<h5> Constants </h5>

In [226]:
Prices_table = 'QUOTEMEDIA/PRICES'
FC_table = 'ZACKS/FC'
FR_table = 'ZACKS/FR'
MT_table = 'ZACKS/MT'
MKTV_table = 'ZACKS/MKTV'
SHRS_table = 'ZACKS/SHRS'
HDM_table = 'ZACKS/HDM'
start_date = '2015-01-01'
end_date = '2022-01-31'
banned_sectors = [5, 13]

sample_size = 200
seed = 81

<h5> Helper Functions </h5>

In [22]:
"""
This code was given by Dr. Boonstra, B., Ph.D. for
University of Chicago FINM 33150 Quandl Options Data Fetching guidelines
"""
def grab_quandl_table(
    table_path,
    avoid_download=False,
    replace_existing=False,
    date_override=None,
    allow_old_file=False,
    **kwargs,
):
    if os.environ['OS'][0:7] == "Windows":
        root_data_dir = os.path.join(os.environ['HOMEPATH'], 'Quandl Data')
    else:
        root_data_dir = os.path.join(os.environ["HOME"], 'Quandl Data')
    data_symlink = os.path.join(root_data_dir, f"{table_path}_latest.zip")
    if avoid_download and os.path.exists(data_symlink):
        print(f"Skipping any possible download of {table_path}")
        return data_symlink

    table_dir = os.path.dirname(data_symlink)
    if not os.path.isdir(table_dir):
        print(f'Creating new data dir {table_dir}')
        os.mkdir(table_dir)

    if date_override is None:
        my_date = datetime.now().strftime("%Y%m%d")
    else:
        my_date = date_override
    data_file = os.path.join(root_data_dir, f"{table_path}_{my_date}.zip")

    if os.path.exists(data_file):
        file_size = os.stat(data_file).st_size
        if replace_existing or not file_size > 0:
            print(f"Removing old file {data_file} size {file_size}")
        else:
            print(
                f"Data file {data_file} size {file_size} exists already, no need to download"
            )
            return data_file

    dl = quandl.export_table(
        table_path, filename = data_file, api_key = 'JbMPn9bSpFPNS7Z7PcZy', **kwargs
    )
    file_size = os.stat(data_file).st_size
    if os.path.exists(data_file) and file_size > 0:
        print(f"Download finished: {file_size} bytes")
        if not date_override:
            try:
                if os.path.exists(data_symlink):
                    print(f"Removing old symlink")
                    os.unlink(data_symlink)
                print(f"Creating symlink: {data_file} -> {data_symlink}")
                os.symlink(
                    data_file, data_symlink,
                )
            except:
                print(f"Symlink Creation Permission Denied")
                data_symlink = data_file
    else:
        print(f"Data file {data_file} failed download")
        return
    return data_symlink if (date_override is None or allow_old_file) else "NoFileAvailable"

In [19]:
"""
This code was given by Dr. Boonstra, B., Ph.D. for
University of Chicago FINM 33150 Quandl Options Data Fetching guidelines
"""
def fetch_quandl_table(table_path, avoid_download=True, **kwargs):
    return pd.read_csv(
        grab_quandl_table(table_path, avoid_download=avoid_download, **kwargs),
        low_memory = False
    )

In [306]:
def choose_tickers(Prices, FC, FR, MT, MKTV, SHRS,
                   start_date, end_date, banned_sectors,
                   sample_size, seed):
    # filter for United States equities
    filter1 = set(MT[MT['country_name'] == 'UNITED STATES']['ticker'].unique())
    # filter for available adj end of day prices data
    filter2 = set(Prices[(Prices['date'] > start_date) & (Prices['date'] < end_date)].dropna()['ticker'].unique())
    # filter for debt/market_cap greater than 0.1 somewhere in the period
    filter3 = set(FR[(FR['per_end_date'] > start_date) & (FR['per_end_date'] < end_date) &
                 (FR['tot_debt_tot_equity'] > 0.1)]['ticker'].unique())
    # filter for automotive and financial sectors
    filter4 = set(FC[(FC['per_end_date'] > start_date) & (FC['per_end_date'] < end_date) &
                 (FC['zacks_sector_code'].apply(lambda x: False if x in banned_sectors else True))]['ticker'].unique())
    ## filtering for feasible calculation
    # filtering for missing tot_debt_tot_equity and ret_invst
    FR_temp = FR[(FR['per_end_date'] > start_date) & (FR['per_end_date'] < end_date)][['ticker',
                                                                                   'tot_debt_tot_equity',
                                                                                   'ret_invst']]
    filter5_FR = set(FR_temp[FR_temp.isnull().any(axis = 1)]['ticker'].unique())
    # filtering for missing mkt_val
    MKTV_temp = MKTV[(MKTV['per_end_date'] > start_date) & (MKTV['per_end_date'] < end_date)][['ticker', 'mkt_val']]
    filter5_MKTV = set(MKTV_temp[MKTV_temp.isnull().any(axis = 1)]['ticker'].unique())
    # filtering for missing values on P/E calculation
    # option 1: diluted EPS
    EPS_Diluted_temp = FC[(FC['per_end_date'] > start_date) & (FC['per_end_date'] < end_date)][['ticker',
                                                                                                'eps_diluted_net']]
    filter5_EPS_Diluted = set(EPS_Diluted_temp[EPS_Diluted_temp.isnull().any(axis = 1)]['ticker'].unique())
    # option 2: basic EPS
    EPS_Basic_1_temp = FC[(FC['per_end_date'] > start_date) & (FC['per_end_date'] < end_date)][['ticker',
                                                                                                'basic_net_eps']]
    filter5_EPS_Basic_1 = set(EPS_Basic_1_temp[EPS_Basic_1_temp.isnull().any(axis = 1)]['ticker'].unique())
    # filtering for 'shares_out' needed for shares out
    EPS_Basic_2_temp = SHRS[(SHRS['per_end_date'] > start_date) & (SHRS['per_end_date'] < end_date)][['ticker',
                                                                                                      'shares_out']]
    filter5_EPS_Basic_2 = set(EPS_Basic_2_temp[EPS_Basic_2_temp.isnull().any(axis = 1)]['ticker'].unique())
    # total filtering for option 2 of basic EPS
    filter5_EPS_Basic = filter5_EPS_Basic_1.union(filter5_EPS_Basic_2, filter5_MKTV)
    # filtering for both options in calculating EPS
    filter5_EPS = filter5_EPS_Diluted.intersection(filter5_EPS_Basic)
    # filtering together
    filter5_inv = filter5_FR.union(filter5_MKTV, filter5_EPS)

    filter = list(filter1.intersection(filter2, filter3, filter4) - filter5_inv)

    np.random.seed(seed)
    return np.random.choice(filter, sample_size, replace = False)

In [313]:
class Ratio_Quantile:
    def __init__(self, ticker, start_date, end_date):
        # inherited data
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        # computed data
        ## Filing Data
        self.Q_df = self.quarterly_data()
        self.A_df = self.annual_data()
        self.filing_df = self.sort_filing_data()
        ## Price data
        self.price_df = self.price_data()
        ## Compiled Data
        self.df = self.final_data()
        ## Ratio Data
        self.ratios = self.compute_ratios()

    def quarterly_data(self):
        global FC
        global FR
        global MKTV
        global SHRS
        start_date = (datetime.strptime(self.start_date, '%Y-%m-%d') + timedelta(days = -120)).strftime('%Y-%m-%d')
        Q1 = FC[(FC['ticker'] == self.ticker) & (FC['per_type'] == 'Q')].set_index('per_end_date').sort_index()\
                    .loc[start_date:self.end_date][['filing_date', 'net_lterm_debt', 'tot_lterm_debt',
                                                    'eps_diluted_net', 'basic_net_eps']]
        Q1['net_lterm_debt'] = Q1['net_lterm_debt'].fillna(0.0)
        Q1['tot_lterm_debt'] = Q1['tot_lterm_debt'].fillna(0.0)
        Q1['eps_diluted_net'] = Q1['eps_diluted_net'].apply(lambda x: 0.001 if x < 0 else x)
        Q1['basic_net_eps'] = Q1['basic_net_eps'].apply(lambda x: 0.001 if x < 0 else x)

        Q2 = FR[(FR['ticker'] == self.ticker) & (FR['per_type'] == 'Q')].set_index('per_end_date').sort_index()\
                    .loc[start_date:self.end_date][['tot_debt_tot_equity', 'ret_invst']]

        Q3 = MKTV[(MKTV['ticker'] == self.ticker) & (MKTV['per_type'] == 'Q')].set_index('per_end_date').sort_index()\
                    .loc[start_date:self.end_date][['mkt_val']]

        Q4 = SHRS[(SHRS['ticker'] == self.ticker) & (SHRS['per_type'] == 'Q')].set_index('per_end_date').sort_index()\
                    .loc[start_date:self.end_date][['shares_out']]

        Q_df = Q1.merge(Q2, how = 'outer', left_index = True, right_index = True)
        Q_df = Q_df.merge(Q3, how = 'outer', left_index = True, right_index = True)
        Q_df = Q_df.merge(Q4, how = 'outer', left_index = True, right_index = True)

        return Q_df.reset_index().set_index('filing_date')

    def annual_data(self):
        global FC
        global FR
        global MKTV
        global SHRS
        start_date = (datetime.strptime(self.start_date, '%Y-%m-%d') + timedelta(days = -120)).strftime('%Y-%m-%d')
        A1 = FC[(FC['ticker'] == self.ticker) & (FC['per_type'] == 'A')].set_index('per_end_date').sort_index()\
                    .loc[start_date:self.end_date][['filing_date', 'net_lterm_debt', 'tot_lterm_debt',
                                                    'eps_diluted_net', 'basic_net_eps']]
        A1['net_lterm_debt'] = A1['net_lterm_debt'].fillna(0.0)
        A1['tot_lterm_debt'] = A1['tot_lterm_debt'].fillna(0.0)
        A1['eps_diluted_net'] = A1['eps_diluted_net'].apply(lambda x: 0.001 if x < 0 else x)
        A1['basic_net_eps'] = A1['basic_net_eps'].apply(lambda x: 0.001 if x < 0 else x)

        A2 = FR[(FR['ticker'] == self.ticker) & (FR['per_type'] == 'A')].set_index('per_end_date').sort_index()\
                    .loc[start_date:self.end_date][['tot_debt_tot_equity', 'ret_invst']]

        A3 = MKTV[(MKTV['ticker'] == self.ticker) & (MKTV['per_type'] == 'A')].set_index('per_end_date').sort_index()\
                    .loc[start_date:self.end_date][['mkt_val']]

        A4 = SHRS[(SHRS['ticker'] == self.ticker) & (SHRS['per_type'] == 'A')].set_index('per_end_date').sort_index()\
                    .loc[start_date:self.end_date][['shares_out']]

        A_df = A1.merge(A2, how = 'outer', left_index = True, right_index = True)
        A_df = A_df.merge(A3, how = 'outer', left_index = True, right_index = True)
        A_df = A_df.merge(A4, how = 'outer', left_index = True, right_index = True)

        return A_df.reset_index().set_index('filing_date')

    def sort_filing_data(self):
        return self.Q_df

    def price_data(self):
        global Prices
        start_date = self.filing_df['per_end_date'][0]
        return Prices[Prices['ticker'] == self.ticker][['date', 'adj_close']].set_index('date').sort_index()\
                    .loc[start_date:self.end_date]

    def final_data(self):
        ret_df = self.price_df.merge(self.filing_df, how = 'outer', left_index = True, right_index = True)\
                    .fillna(method = 'ffill')

        ret_df['per_end_date'] = ret_df['per_end_date'].fillna(method = 'bfill')

        base_price = []
        for date in ret_df['per_end_date']:
            base_price.append(ret_df.loc[:date]['adj_close'][-1])

        ret_df['base_price'] = pd.Series(base_price, index = ret_df.index)

        return ret_df.loc[self.start_date:self.end_date]

    def compute_ratios(self):
        df = self.df.copy()
        df['debt to market cap'] = df['tot_debt_tot_equity'] * df['base_price'] /df['adj_close']
        df['M'] = df['mkt_val'] / df['base_price'] * df['adj_close']
        df['return on investment'] = df['ret_invst'] / (df['lterm_debt'] + df['M'])
        df['price to earnings'] = df['M'] / (df['eps'] * df['shares_out'])

        return df[['debt to market cap', 'return on investment', 'price to earnings']]

<h2> 2. Data </h2>

<h3> 2-1. Raw Data </h3>

In [21]:
Prices = fetch_quandl_table(Prices_table, avoid_download = False)
FC = fetch_quandl_table(FC_table, avoid_download = False)
FR = fetch_quandl_table(FR_table, avoid_download = False)
MT = fetch_quandl_table(MT_table, avoid_download = False)
MKTV = fetch_quandl_table(MKTV_table, avoid_download = False)
SHRS = fetch_quandl_table(SHRS_table, avoid_download = False)
HDM = fetch_quandl_table(HDM_table, avoid_download = False)

Data file \Users\kwhyu\Quandl Data\QUOTEMEDIA/PRICES_20230124.zip size 1372443502 exists already, no need to download
Data file \Users\kwhyu\Quandl Data\ZACKS/FC_20230124.zip size 175785898 exists already, no need to download
Data file \Users\kwhyu\Quandl Data\ZACKS/FR_20230124.zip size 36375570 exists already, no need to download
Data file \Users\kwhyu\Quandl Data\ZACKS/MT_20230124.zip size 992725 exists already, no need to download
Data file \Users\kwhyu\Quandl Data\ZACKS/MKTV_20230124.zip size 6693234 exists already, no need to download
Download finished: 5100081 bytes
Creating symlink: \Users\kwhyu\Quandl Data\ZACKS/SHRS_20230124.zip -> \Users\kwhyu\Quandl Data\ZACKS/SHRS_latest.zip
Symlink Creation Permission Denied
Download finished: 1099253 bytes
Creating symlink: \Users\kwhyu\Quandl Data\ZACKS/HDM_20230124.zip -> \Users\kwhyu\Quandl Data\ZACKS/HDM_latest.zip
Symlink Creation Permission Denied


<h3> 2-2. Data Processing </h3>

In [307]:
tickers = choose_tickers(Prices, FC, FR, MT, MKTV, SHRS,
                         start_date, end_date, banned_sectors,
                         sample_size, seed)

In [314]:
test1 = Ratio_Quantile(tickers[0], start_date, end_date)

In [315]:
test1.ratios

,debt to market cap,return on investment,price to earnings
2015-01-02,1.437949,0.000055,50.101142
2015-01-05,1.477526,0.000056,48.759147
2015-01-06,1.469820,0.000056,49.014765
2015-01-07,1.457657,0.000056,49.423754
2015-01-08,1.444583,0.000055,49.871085
...,...,...,...
2022-01-25,2.708404,0.000057,49.098980
2022-01-26,2.718932,0.000058,48.908859
2022-01-27,2.712342,0.000057,49.027685
2022-01-28,2.680509,0.000057,49.609933


<h3> 2-3. Data Description </h3>

<h2> 3. Analysis </h2>